In [2]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, snoop, matplotlib
from pprint import pprint as print

matplotlib.style.use("seaborn-whitegrid")
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)
plt.rcParams["figure.figsize"] = (15, 10)

%load_ext snoop
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_format = 'svg' 
# show all output not only the last result
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("/Users/glebsokolov/HeadRepo/DataSets/test.csv")


In [3]:
df

,time,user_id,type,amount,target,feature1,feature2,events_ordinal_number,second_event_time,loan_ordinal_number,events_time_diff,previous_loans_max_amount
0,2001-01-01 00:00:00,443,loanRequest,17017,NaN,1,1,1,2001-11-22 23:00:00,1,NaN,NaN
1,2001-01-01 01:00:00,920,loanRequest,13304,NaN,1,0,1,2002-02-16 08:00:00,1,NaN,NaN
2,2001-01-01 02:00:00,416,loanRequest,13047,NaN,1,0,1,2002-02-18 10:00:00,1,NaN,NaN
3,2001-01-01 03:00:00,382,loanRequest,14201,NaN,0,1,1,2002-02-17 08:00:00,1,NaN,NaN
4,2001-01-01 04:00:00,258,loan,14621,0.0,0,1,1,2001-12-03 11:00:00,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2002-02-21 11:00:00,738,loanRequest,14195,NaN,0,0,8,2002-02-21 11:00:00,1,2 days 02:00:00,NaN
9996,2002-02-21 12:00:00,881,loan,13940,0.0,0,1,14,2002-02-21 12:00:00,2,52 days 15:00:00,14265.0
9997,2002-02-21 13:00:00,72,loan,14325,1.0,1,1,11,2002-02-21 13:00:00,3,27 days 12:00:00,16306.0
9998,2002-02-21 14:00:00,563,loan,15718,0.0,1,0,12,2002-02-21 14:00:00,5,3 days 15:00:00,17170.0
